In [1]:
!pip install yfinance --upgrade --no-cache-dir -q

In [2]:
import pandas as pd
import numpy as np
import yfinance
import requests
import lxml
import yfinance as yf
from torch.nn.functional import softmax
from tqdm import tqdm


# Function to download and process financial data using yfinance directly
def financial_dataset(stock, num_of_labels=2, cutoff=0.25,
                      start_date="2010-01-01", end_date="2021-01-01"):
    ''' Downloads financial data for a stock and processes it in the desired format. '''
    
    # Parameter value check
    if (num_of_labels < 2 or num_of_labels > 3):
        return print('Number of labels can be either 2 or 3')
    
    # Download financial data using yfinance
    fin_data = yf.download(stock, start=start_date, end=end_date)
    
    print(f"{stock} financial dataframe dimensions: ", fin_data.shape)
    
    # Initialize price_change column 
    fin_data['Price_change'] = 1
    fin_data['date'] = 0
    dates = fin_data.index
    yesterday = str(dates[0].date())

    # Calculate price changes
    for date in dates[1:]:
        today = str(date.date())

        yesterday_pr = fin_data.loc[yesterday, 'Close']
        today_pr = fin_data.loc[today, 'Close']
        diff = 100 * (today_pr - yesterday_pr) / yesterday_pr

        if (num_of_labels == 3):
            if (diff > cutoff):
                price_change = +1
            elif (diff < -cutoff):
                price_change = -1
            else:
                price_change = 0 
        elif (num_of_labels == 2):
            if (diff > 0): 
                price_change = +1
            elif (diff <= 0):
                price_change = -1 
                                                                                                       
        yesterday = today
        fin_data.loc[today, 'Price_change'] = price_change
        fin_data.loc[today, 'date'] = today

    # Summary of changes
    incr = fin_data[fin_data['Price_change'] == 1].shape[0]
    decr = fin_data[fin_data['Price_change'] == -1].shape[0]
    stable = fin_data[fin_data['Price_change'] == 0].shape[0]
    print(f'Positive changes: {incr}')
    print(f'Negative changes: {decr}')
    print(f'No changes: {stable}')

    # Drop unnecessary columns
    fin_data = fin_data.drop(columns=['Low', 'High', 'Adj Close'], axis=1)
        
    return fin_data

# Function to read financial news from two different datasets
def read_news(stock):
    def read_rph(stock):
        ''' Reads news relevant to 'stock' from the "raw_partner_headlines.csv" file. '''
        csv_path = 'Financial_News/raw_partner_headlines.csv'
        arp = pd.read_csv(csv_path)
        arp = arp.drop(columns=['Unnamed: 0', 'url', 'publisher'], axis=1)
        arp['date'] = arp['date'].apply(lambda x: x.split(' ')[0])
        news = arp[arp['stock'] == stock]
        print(f"The bot found {news.shape[0]} headlines from raw_partner_headlines.csv regarding {stock} stock")
        return news

    def read_arp(stock):
        ''' Reads news relevant to 'stock' from the "analyst_rating_processed.csv" file. '''
        csv_path = 'Financial_News/analyst_ratings_processed.csv'
        arp = pd.read_csv(csv_path)
        arp = arp.drop(columns=['Unnamed: 0'], axis=1)
        arp['date'] = arp['date'].apply(lambda x: str(x).split(' ')[0])
        arp.rename({'title': 'headline'}, axis=1, inplace=True)
        news = arp[arp['stock'] == stock]
        print(f"The bot found {news.shape[0]} headlines from analyst_ratings_processed.csv regarding {stock} stock")
        return news
    
    arp = read_arp(stock)
    rph = read_rph(stock)
    news = pd.concat([rph, arp], ignore_index=True)
    print(f"The bot found {news.shape[0]} headlines in total regarding {stock} stock")
    return news

# Function to merge financial data and news headlines
def merge_fin_news(df_fin, df_news, how='inner'):
    ''' Merges financial data with news data based on the date column '''
    merged_df = df_fin.merge(df_news, on='date', how=how)
    merged_df = merged_df[['date', 'stock', 'Open', 'Close', 'Volume', 'headline', 'Price_change']]
    return merged_df

# Function to analyze sentiment in the news headlines using a pre-trained FinBert model
def sentim_analyzer(df, tokenizer, model):
    ''' Runs sentiment analysis on headlines and adds sentiment columns to the dataframe '''
    for i in tqdm(df.index):
        try:
            headline = df.loc[i, 'headline']
        except:
            return print('\'headline\' column might be missing from dataframe')
        
        input = tokenizer(headline, padding=True, truncation=True, return_tensors='pt')
        output = model(**input)
        predictions = softmax(output.logits, dim=-1)
        
        df.loc[i, 'Positive'] = predictions[0][0].tolist()
        df.loc[i, 'Negative'] = predictions[0][1].tolist()
        df.loc[i, 'Neutral'] = predictions[0][2].tolist()
    
    try:
        df = df[['date', 'stock', 'Open', 'Close', 'Volume', 'headline', 'Positive', 'Negative', 'Neutral', 'Price_change']]
    except:
        pass
    return df

# Function to merge sentiment scores by date
def merge_dates(df):
    ''' Aggregates sentiment scores by date, taking the average for each date '''
    dates_in_df = df['date'].unique()
    
    # Create an empty DataFrame with the required columns
    new_df = pd.DataFrame(columns=['date', 'stock', 'Open', 'Close', 'Volume', 
                                   'Positive', 'Negative', 'Neutral', 'Price_change'])
    
    for date in dates_in_df:
        # Filter data for the current date
        sub_df = df[df['date'] == date]
        
        # Calculate average sentiment scores
        avg_positive = sub_df['Positive'].mean()
        avg_negative = sub_df['Negative'].mean()
        avg_neutral = sub_df['Neutral'].mean()
        
        # Get other values (assuming they are constant per date)
        stock = sub_df.iloc[0]['stock']
        open_price = sub_df.iloc[0]['Open']
        close_price = sub_df.iloc[0]['Close']
        volume = sub_df.iloc[0]['Volume']
        price_change = sub_df.iloc[0]['Price_change']
        
        # Create a new row as a DataFrame
        new_row = pd.DataFrame({
            'date': [date],
            'stock': [stock],
            'Open': [open_price],
            'Close': [close_price],
            'Volume': [volume],
            'Positive': [avg_positive],
            'Negative': [avg_negative],
            'Neutral': [avg_neutral],
            'Price_change': [price_change]
        })
        
        # Concatenate the new row to new_df
        new_df = pd.concat([new_df, new_row], ignore_index=True)
    
    print(f"Dataframe now contains sentiment scores for {new_df.shape[0]} different dates.")
    
    return new_df

In [3]:
!pip install transformers -q

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

c:\Users\Asus\anaconda3\envs\venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
#['AAPL', 'MSFT','AMZN','GOOG', 'TSLA', 'NVDA',
#stocks =         ['BRK', 'FB', 'JPM',
        #  'UNH', 'JNJ', 'PG', 'V', 'HD','BAC', 'XOM', 'MA', 'DIS', 'PFE', 'CVX',
         # 'ABBV', 'KO', 'CSCO', 'AVGO', 'VZ', 'PEP', 'WMT', 
stocks =[
    'AAPL',  # Apple
    'MSFT',  # Microsoft
    'GOOG',  # Alphabet (Class C)
    'GOOGL', # Alphabet (Class A)
    'AMZN',  # Amazon
    'NVDA',  # Nvidia
    'TSLA',  # Tesla
    'BRK', # Berkshire Hathaway (Class B)
    'META',  # Meta Platforms (formerly Facebook)
    'V',     # Visa
    'UNH',   # UnitedHealth Group
    'XOM',   # ExxonMobil
    'JNJ',   # Johnson & Johnson
    'JPM',   # JPMorgan Chase
    'WMT',   # Walmart
    'PG',    # Procter & Gamble
    'MA',    # Mastercard
    'LLY',   # Eli Lilly
    'HD',    # Home Depot
    'CVX',   # Chevron
    'MRK',   # Merck
    'ABBV',  # AbbVie
    'PEP',   # PepsiCo
    'KO',    # Coca-Cola
    'AVGO',  # Broadcom
    'COST',  # Costco Wholesale
    'MCD',   # McDonald's
    'BAC',   # Bank of America
    'PFE',   # Pfizer
    'ADBE',  # Adobe
    'CRM',   # Salesforce
    'CSCO',  # Cisco Systems
    'TXN',   # Texas Instruments
    'TMO',   # Thermo Fisher Scientific
    'NEE'    # NextEra Energy
]

In [6]:
# Read finances and news for all stocks in the list and create a big dataframe
for stock in stocks:
    # Exception handling for META to include FB news
    if stock == 'META':
        news = handle_meta_fb(pd.DataFrame(), stock)
    else:
        news = read_news(stock)
    
    finances = financial_dataset(stock, num_of_labels=2)
    merged_df = merge_fin_news(finances, news)
    
    print(f"Performing sentiment analysis for stock {stock}")
    
    # Pass the correct tokenizer and model
    sentimentized_df = sentim_analyzer(merged_df, tokenizer=tokenizer, model=model)
    
    # Average sentiment for each unique date
    sentimentized_df = merge_dates(sentimentized_df)
    
    # Add results to the big dataframe
    big_df = pd.concat([big_df, sentimentized_df], axis=0, ignore_index=True)

The bot found 469 headlines from analyst_ratings_processed.csv regarding AAPL stock
The bot found 32 headlines from raw_partner_headlines.csv regarding AAPL stock
The bot found 501 headlines in total regarding AAPL stock


[*********************100%***********************]  1 of 1 completed
C:\Users\Asus\AppData\Local\Temp\ipykernel_67824\4278251462.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2010-01-05' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  fin_data.loc[today, 'date'] = today


AAPL financial dataframe dimensions:  (2769, 6)
Positive changes: 1467
Negative changes: 1302
No changes: 0
Performing sentiment analysis for stock AAPL


  0%|          | 0/462 [00:00<?, ?it/s]


TypeError: 'NoneType' object is not callable

In [ ]:
big_df

In [ ]:
big_df.to_csv('Datasets/sp500_news.csv')

In [ ]:
dates = big_df.date.unique()

In [ ]:
len(dates)

In [ ]:
big_df.shape

In [ ]:
sp500 = yf.download("^GSPC", start="2010-01-01", end="2021-01-01")

In [ ]:
sp500